In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv

# ChromeDriver 경로 설정
chrome_driver_path = "C:/Users/chica/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

# WebDriver 옵션 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

# WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)  # 대기 시간 5초 설정

# 페이지 불러오기
driver.get('https://www.wanted.co.kr/wdlist/532?country=kr&job_sort=job.latest_order&years=-1&locations=all')  # 직무 선택된 URL

# 자동 스크롤 기능 구현
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로드 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(5)  # 추가 대기 시간 (페이지가 완전히 로드될 때까지 기다림)

# 스크롤을 끝까지 내린다
scroll_to_bottom()

# 공고 리스트의 모든 링크를 저장 (스크롤 후 모든 공고 리스트 가져오기)
# 원래 알고리즘: 카드 번호 순서대로 들어가기
# 원래 알고리즘 문제점: 전체 로드 해놔도 인식을 못함. 맨 위에 몇개만 가져오고 오류.
# 바꾼 알고리즘: 전체 로드 후, 카드의 url 저장해서 하나하나 들어가기

job_links = driver.find_elements(By.CSS_SELECTOR, 'ul.List_List__Ni_dK li a')
job_urls = [job.get_attribute('href') for job in job_links]  # 모든 공고의 링크를 저장

# # 페이지 상단으로 다시 스크롤
# driver.execute_script("window.scrollTo(0, 0);")

try:
    # 수집한 데이터를 CSV로 작성할 때, UTF-8 인코딩 확인
    with open("../데이터/원티드 채용공고_물류 및 무역.csv", 'w', newline='', encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        # writer.writerow(['채용공고명', '포지션 상세', '주요 업무', '자격 요건', '우대사항', '혜택 및 복지', '채용 전형', '위치', '마감일', '태그', '회사명'])  # CSV 헤더 설정
        writer.writerow(['채용공고명', '포지션 상세', '주요 업무', '자격 요건', '우대사항', '혜택 및 복지', '채용 전형', '태그', '마감일', '면접리뷰', '회사위치', '회사명'])

        for job_url in job_urls:
            try:
                driver.get(job_url)
                time.sleep(2)  # 페이지 로딩 대기

                # try:
                #     tags_ul = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[2]/ul')  # ul 리스트에서 모든 li 태그 선택
                #     tags = [tag.text for tag in tags_ul]  
                #     tags_text = ', '.join(tags)  
                # except:
                #     tags_text = ""
                # 태그 정보 추출
                try:
                    tags_ul = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[2]/ul/li')
                    tags = [tag.text for tag in tags_ul]  
                    tags_text = ', '.join(tags)  # 태그를 ,로 연결
                except:
                    tags_text = ""

                try:
                    location = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/span[2]').text
                except:
                    location = ""

                try:
                    more_button = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/button')
                    more_button.click()
                    time.sleep(1)  # 페이지 로딩 대기
                except:
                    print(f"No '상세 정보 더 보기' button for job: {job_url}. Continuing...")

                # 필요한 정보 수집 (이게 공고마다 다 달라서 없으면 빈문자열로 받도록 처리헸습니다)
                # 더 필요한 항목이 있다면 추가하여 수정 후 push 부탁드립니다
                try:
                    title = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/h1').text
                except:
                    title = ""

                try:
                    details = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/p/span').text
                except:
                    details = ""

                try:
                    main_tasks = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[1]/p').text
                except:
                    main_tasks = ""

                try:
                    qualifications = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[2]/p').text
                except:
                    qualifications = ""

                try:
                    preferred = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[3]/p').text
                except:
                    preferred = ""

                try:
                    benefits = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[4]/p').text
                except:
                    benefits = ""

                try:
                    process = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[1]/div/div[5]/p').text
                except:
                    process = ""

                try:
                    deadline = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[3]/span').text
                except:
                    deadline = ""

                try:
                    company_name = driver.find_element(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/header/div/div[1]/a').text
                except:
                    company_name = ""
                
                # 면접 리뷰 정보 추출
                try:
                    interview_review_elements = driver.find_elements(By.XPATH, '//*[@id="__next"]/main/div[1]/div/section/section/article[4]/ul/li/span')
                    interview_reviews = [review.text for review in interview_review_elements]
                    interview_review_text = ', '.join(interview_reviews)  # ,로 연결
                except:
                    interview_review_text = ""

                # # 수집한 데이터를 CSV 파일에 작성
                # writer.writerow([title, details, main_tasks, qualifications, preferred, benefits, process, location, deadline, tags_text, company_name])
                # 수집한 데이터를 CSV 파일에 작성
                writer.writerow([title, details, main_tasks, qualifications, preferred, benefits, process, tags_text, deadline, interview_review_text, location, company_name])

            except Exception as e:
                print(f"Error processing job {job_url}: {e}")

finally:
    driver.quit()


In [6]:
# 직무별 채용공고 개수 확인
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# ChromeDriver 경로 설정
chrome_driver_path = "C:/Users/chica/OneDrive/바탕 화면/chromedriver-win64/chromedriver.exe"

# WebDriver 옵션 설정
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36")

# WebDriver 초기화
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(3)

# 페이지 불러오기
driver.get('https://www.wanted.co.kr/wdlist/532?country=kr&job_sort=job.latest_order&years=-1&locations=all')  # 직무 선택된 url

# 자동 스크롤 기능 구현
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 페이지 로드 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# 스크롤을 끝까지 내린다
scroll_to_bottom()

# 카드 총 개수 확인
cards = driver.find_elements(By.CSS_SELECTOR, 'li.Card_Card__WdaEk')  # 카드 클래스 선택
total_cards = len(cards)

print(f"총 카드 개수: {total_cards}")

driver.quit()


총 카드 개수: 88
